
# Biofabrication — Chapter 4: Interactive Python Exercises (Student Notebook, v3)

Run the **Setup** cell first. Each exercise includes a short description, a LaTeX-rendered equation, and interactive controls (sliders/dropdowns).


In [ ]:
# @title Setup (helpers, units, widgets)
import math, numpy as np
from IPython.display import display, Markdown
import ipywidgets as widgets

# Units
mm_to_m = 1e-3
um_to_m = 1e-6
cm3_to_m3 = 1e-6

display(Markdown("**Ready.** Widgets loaded."))


## 1) Photocuring Dose → Exposure Time

Compute exposure time from required dose and intensity.

$$
\text{Dose} = \text{Intensity}\cdot t
\quad\Rightarrow\quad
t = \frac{\text{Dose}}{\text{Intensity}}\;.
$$


In [ ]:
# @title Photocuring dose calculator (with slider)
dose = widgets.FloatSlider(value=50.0, min=1.0, max=200.0, step=1.0, description='Dose (mJ/cm²):')
intensity = widgets.FloatSlider(value=10.0, min=0.5, max=100.0, step=0.5, description='Intensity (mW/cm²):')
out = widgets.Output()

def update_exposure(*args):
    with out:
        out.clear_output()
        I = intensity.value
        D = dose.value
        t = D / I
        display(Markdown(f"**Exposure time:** `{t:.2f} s`"))
        display(Markdown(f"If intensity ×5 → `{(D/(5*I)):.2f} s` (watch phototoxicity/heating)."))

dose.observe(update_exposure, 'value')
intensity.observe(update_exposure, 'value')

display(widgets.VBox([dose, intensity, out]))
update_exposure()


## 2) Layer-by-Layer Build Time

For height $H$, layer thickness $\Delta z$, and per-layer time $t_\ell$:

$$
T = \left\lceil \frac{H}{\Delta z} \right\rceil\, t_\ell\;.
$$


In [ ]:
# @title Build time calculator (layers)
H = widgets.FloatSlider(value=12.0, min=1.0, max=200.0, step=1.0, description='Height H (mm):')
dz = widgets.FloatSlider(value=0.10, min=0.01, max=1.0, step=0.01, readout_format='.2f', description='Layer Δz (mm):')
tL = widgets.FloatSlider(value=30.0, min=1.0, max=240.0, step=1.0, description='Time/layer (s):')
out2 = widgets.Output()

def upd2(*_):
    with out2:
        out2.clear_output()
        layers = math.ceil(H.value/dz.value)
        T = layers*tL.value
        display(Markdown(f"**Layers:** `{layers}` — **Total time:** `{T/60:.1f} min`"))
for w in (H,dz,tL): w.observe(upd2,'value')
display(widgets.VBox([H,dz,tL,out2])); upd2()


## 3) SLA Exposure Budget (resolution vs speed)

Total exposure for an SLA print with layer thickness $\Delta z$ and exposure per layer $t_e$:

$$
T_{\text{exp}} = \left\lceil \frac{H}{\Delta z} \right\rceil\, t_e\;.
$$


In [ ]:
# @title SLA exposure budget
H3 = widgets.FloatSlider(value=30.0, min=1.0, max=200.0, step=1.0, description='Height H (mm):')
dz3 = widgets.FloatSlider(value=0.05, min=0.01, max=0.50, step=0.01, readout_format='.2f', description='Layer Δz (mm):')
te  = widgets.FloatSlider(value=20.0, min=1.0, max=240.0, step=1.0, description='Exposure/layer (s):')
out3 = widgets.Output()

def upd3(*_):
    with out3:
        out3.clear_output()
        layers = math.ceil(H3.value/dz3.value)
        Texpo = layers*te.value
        display(Markdown(f"**Layers:** `{layers}` — **Total exposure:** `{Texpo/3600:.2f} h`"))
for w in (H3,dz3,te): w.observe(upd3,'value')
display(widgets.VBox([H3,dz3,te,out3])); upd3()


## 4) Mass of Printed Implant (Infill)

Mass:
$$
m = V\, \rho\, f\;,
$$
where $f$ is the infill fraction (0–1).


In [ ]:
# @title Implant mass with infill
V   = widgets.FloatSlider(value=2.0, min=0.1, max=200.0, step=0.1, description='Volume (cm³):')
rho = widgets.FloatSlider(value=1.1, min=0.2, max=3.0,  step=0.1, description='Density (g/cm³):')
f   = widgets.FloatSlider(value=0.40, min=0.05, max=1.0,  step=0.05, readout_format='.2f', description='Infill fraction:')
out4 = widgets.Output()

def upd4(*_):
    with out4:
        out4.clear_output()
        m = V.value * rho.value * f.value
        display(Markdown(f"**Mass:** `{m:.3f} g`"))
for w in (V,rho,f): w.observe(upd4,'value')
display(widgets.VBox([V,rho,f,out4])); upd4()


## 5) Oxygen Diffusion & Viable Thickness (1D steady-state)

Solve on half-thickness $L$ with $c(0)=c_0$ and $\frac{dc}{dx}\big|_{x=L}=0$ for

$$
D\,\frac{d^2 c}{dx^2} - k\,c = 0,
$$

and find the largest $L$ with $\min c(x) \ge c_{\min}$.


In [ ]:
# @title Viable thickness (diffusion–reaction) with sliders
c0   = widgets.FloatSlider(value=1.0, min=0.01, max=1.0,  step=0.01, description='c0 (norm):')
cmin = widgets.FloatSlider(value=0.10, min=0.01, max=0.90, step=0.01, description='c_min:')
D    = widgets.FloatLogSlider(value=3e-9, base=10, min=-10, max=-7, step=0.01, description='D (m²/s):')
k    = widgets.FloatLogSlider(value=1e-2, base=10, min=-4,  max=0,  step=0.01, description='k (1/s):')
out5 = widgets.Output()

def _viable_half_thickness(c0_val, cmin_val, D_val, k_val):
    def min_c(L):
        N = max(50, int(200*L/(200e-6)))
        x = np.linspace(0, L, N+1)
        dx = x[1]-x[0]
        a = D_val/(dx*dx); b = k_val
        A = np.zeros((N+1, N+1)); rhs = np.zeros(N+1)
        A[0,0]=1.0; rhs[0]=c0_val
        for i in range(1,N):
            A[i,i-1]=a; A[i,i]=-2*a-b; A[i,i+1]=a
        A[N,N]=1.0; A[N,N-1]=-1.0
        sol = np.linalg.solve(A, rhs)
        return sol.min()
    lo, hi = 10e-6, 2e-3
    for _ in range(30):
        mid = 0.5*(lo+hi)
        if min_c(mid) >= cmin_val: lo = mid
        else: hi = mid
    return lo

def upd5(*_):
    with out5:
        out5.clear_output()
        L = _viable_half_thickness(c0.value, cmin.value, D.value, k.value)
        display(Markdown(f"**Max half-thickness** ≈ `{L*1e6:.0f} µm`  → **full** ≈ `{2*L*1e6:.0f} µm`"))
for w in (c0,cmin,D,k): w.observe(upd5,'value')
display(widgets.VBox([c0,cmin,D,k,out5])); upd5()


## 6) Effective Diffusivity in Porous Hydrogels

$$
D_e = D_0\, \frac{\varepsilon}{\tau}\;.
$$


In [ ]:
# @title Effective diffusivity
D0  = widgets.FloatLogSlider(value=3e-9, base=10, min=-10, max=-7, step=0.01, description='D0 (m²/s):')
eps = widgets.FloatSlider(value=0.80, min=0.10, max=0.99, step=0.01, description='Porosity ε:')
tau = widgets.FloatSlider(value=2.00, min=1.00, max=5.00, step=0.10, description='Tortuosity τ:')
out6 = widgets.Output()

def upd6(*_):
    with out6:
        out6.clear_output()
        De = D0.value * eps.value / tau.value
        display(Markdown(f"**De** = `{De:.2e} m²/s`; if τ doubles → `{D0.value*eps.value/(2*tau.value):.2e} m²/s`"))
for w in (D0,eps,tau): w.observe(upd6,'value')
display(widgets.VBox([D0,eps,tau,out6])); upd6()


## 7) Extrusion Flow & Wall Shear Stress (Hagen–Poiseuille)

$$
Q = \frac{\pi r^4 P}{8\eta L}, \qquad
\tau_w = \frac{4\eta Q}{\pi r^3}\;.
$$


In [ ]:
# @title Extrusion flow and shear
r   = widgets.FloatSlider(value=150.0, min=25.0, max=500.0, step=5.0, description='Radius (µm):')
Lnoz= widgets.FloatSlider(value=5.0,   min=1.0,  max=50.0, step=0.5, description='Length (mm):')
eta = widgets.FloatLogSlider(value=0.1, base=10, min=-2, max=1, step=0.01, description='η (Pa·s):')
P   = widgets.FloatSlider(value=150.0, min=5.0,  max=500.0, step=5.0, description='Pressure (kPa):')
out7 = widgets.Output()

def upd7(*_):
    with out7:
        out7.clear_output()
        rr = r.value*1e-6; LL = Lnoz.value*1e-3; PP = P.value*1e3
        Q  = math.pi*rr**4*PP/(8.0*eta.value*LL)
        tau= 4.0*eta.value*Q/(math.pi*rr**3)
        display(Markdown(f"**Q:** `{Q*1e9:.2f} nL/s` — **wall shear τ:** `{tau:.1f} Pa` (≤ ~200 Pa?)"))
for w in (r,Lnoz,eta,P): w.observe(upd7,'value')
display(widgets.VBox([r,Lnoz,eta,P,out7])); upd7()


## 8) Composite Modulus (Voigt/Reuss bounds)

Upper (iso-strain) and lower (iso-stress) bounds:

$$
E_V = \sum v_i E_i, \qquad
E_R = \left(\sum \frac{v_i}{E_i}\right)^{-1}.
$$


In [ ]:
# @title Composite modulus bounds
v_PCL = widgets.FloatSlider(value=0.60, min=0.0, max=1.0, step=0.05, description='v_PCL:')
E_PCL = widgets.FloatLogSlider(value=300e6, base=10, min=3, max=10, step=0.01, description='E_PCL (Pa):')
E_h   = widgets.FloatLogSlider(value=0.5e6, base=10, min=3, max=8,  step=0.01, description='E_hydrogel (Pa):')
out8  = widgets.Output()

def upd8(*_):
    with out8:
        out8.clear_output()
        v1, v2 = v_PCL.value, 1-v_PCL.value
        Ev = v1*E_PCL.value + v2*E_h.value
        Er = 1.0/(v1/E_PCL.value + v2/E_h.value)
        display(Markdown(f"**Voigt upper:** `{Ev/1e6:.2f} MPa` — **Reuss lower:** `{Er/1e6:.4f} MPa`"))
for w in (v_PCL,E_PCL,E_h): w.observe(upd8,'value')
display(widgets.VBox([v_PCL,E_PCL,E_h,out8])); upd8()


## 9) (Bonus) 4D Thermoresponsive Bilayer Bending

Approximate curvature for mismatch strain $\epsilon = \Delta\alpha\,\Delta T$.


In [ ]:
# @title Bilayer curvature (interactive)
E1 = widgets.FloatLogSlider(value=10e3, base=10, min=2, max=7, step=0.01, description='E1 (Pa):')
E2 = widgets.FloatLogSlider(value=50e3, base=10, min=2, max=7, step=0.01, description='E2 (Pa):')
t1 = widgets.FloatSlider(value=0.5, min=0.1, max=2.0, step=0.05, description='t1 (mm):')
t2 = widgets.FloatSlider(value=0.5, min=0.1, max=2.0, step=0.05, description='t2 (mm):')
dalpha = widgets.FloatLogSlider(value=2e-3, base=10, min=-5, max=-1, step=0.01, description='Δα (1/°C):')
dT = widgets.FloatSlider(value=12.0, min=-20.0, max=40.0, step=0.5, description='ΔT (°C):')
out9 = widgets.Output()

def curvature(E1v,E2v,t1v,t2v,da,dT):
    n = E2v/E1v; m = (t2v*1e-3)/(t1v*1e-3)
    eps = da*dT
    return (6*eps)/((t1v*1e-3)*(1+4*m+6*m**2+4*m**3+m**4)*(1+n*m))

def upd9(*_):
    with out9:
        out9.clear_output()
        kappa = curvature(E1.value,E2.value,t1.value,t2.value,dalpha.value,dT.value)
        R = float('inf') if kappa==0 else 1.0/kappa
        display(Markdown(f"**Curvature κ:** `{kappa:.3f} 1/m` — **Radius R:** `{R:.2f} m`"))
for w in (E1,E2,t1,t2,dalpha,dT): w.observe(upd9,'value')
display(widgets.VBox([E1,E2,t1,t2,dalpha,dT,out9])); upd9()